In [1]:
import gpytorch
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms

gpytorch.functions.use_toeplitz = False

In [2]:
num_few_shot_labels = 5
class FeatureExtractor(nn.Sequential):
    
    def __init__(self):
        super(FeatureExtractor, self).__init__(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2))
        
class Bottleneck(nn.Sequential):
    
    def __init__(self):
        super(Bottleneck, self).__init__(nn.Linear(64*7*7, 128),
                                         nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128, 128),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128,64),
                                 nn.BatchNorm1d(64))

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bottleneck = Bottleneck()
        self.final_layer = nn.Sequential(
                                 nn.ReLU(),
                                 nn.Linear(64,1319-25+num_few_shot_labels))
    
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        classification = self.final_layer(bottlenecked_features)
        return classification
        

In [3]:
train_omni = torchvision.datasets.ImageFolder('/scratch/bw462/omni_data/general', transform=transforms.Compose([
                        transforms.Scale((28,28)),
                        transforms.ToTensor()
                   ])) 
# have general with all but 25 classes
# Then have way5 with 5 lcasses and way20 with 20 classes
# this is on nikola 01 and 04


In [4]:
train_data_loader = torch.utils.data.DataLoader(train_omni, shuffle=True, pin_memory=True, batch_size=256)

In [5]:
criterion = nn.CrossEntropyLoss().cuda()

In [6]:
model = LeNet().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [7]:
num_epochs = 5
if num_epochs > 0:
    for i in range(num_epochs):
        for x, y in train_data_loader:
            optimizer.zero_grad()
            x = Variable(x[:,0,:,:].unsqueeze(1).cuda())
            y = Variable(y.cuda())
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
        print("Loss: %.3f" % loss.data[0])
    torch.save(model.state_dict(), '/scratch/bw462/lenet_omni_oneshot.dat')
else:
    model.load_state_dict(torch.load('/scratch/bw462/lenet_omni_oneshot.dat'))
"""
model.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)
avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = model(Variable(test_batch_x).cuda()).max(-1)[1]
    test_batch_y = Variable(test_batch_y).cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.
print('Accuracy: %.4f' % (avg / i))
"""

Loss: 7.048
Loss: 6.953
Loss: 6.831
Loss: 6.605
Loss: 6.318


"\nmodel.eval()\ntest_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)\navg = 0.\ni = 0.\nfor test_batch_x, test_batch_y in test_data_loader:\n    predictions = model(Variable(test_batch_x).cuda()).max(-1)[1]\n    test_batch_y = Variable(test_batch_y).cuda()\n    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]\n    i += 1.\nprint('Accuracy: %.4f' % (avg / i))\n"

In [8]:
list(model.bottleneck.modules())[-1].weight.data.fill_(1)
None

In [9]:
from gpytorch.kernels import RBFKernel, GridInterpolationKernel

class DeepKernel(gpytorch.Module):
    def __init__(self, model):
        super(DeepKernel, self).__init__()
        self.feature_extractor = model.feature_extractor
        self.bottleneck = model.bottleneck
        self.gp_layer = GPLayer()
        
    def forward(self, x):
        features = self.feature_extractor(x[:,0,:,:].unsqueeze(1))
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        gp_output = self.gp_layer(0.1*bottlenecked_features)
        return gp_output
    
    
class LatentFunction(gpytorch.AdditiveGridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=256, grid_bounds=[(-7, 7)],
                                             n_components=64, mixing_params=False, sum_output=False)
        cov_module = RBFKernel()
        cov_module.initialize(log_lengthscale=2)
        self.cov_module = cov_module
        
    def forward(self, x):
        mean = Variable(x.data.new(len(x)).zero_())
        covar = self.cov_module(x)
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

    
class GPLayer(gpytorch.GPModel):
    def __init__(self, n_dims=64):
        super(GPLayer, self).__init__(gpytorch.likelihoods.SoftmaxGaussianLikelihood(n_features=64,
                                                                                     n_classes=1319-25+num_few_shot_labels,
                                                                                     rank=5))
        self.latent_function = LatentFunction()
    
    def forward(self, x):
        res = self.latent_function(x)
        return res
    

In [10]:
deep_kernel = DeepKernel(model).cuda()
gp_data_loader = torch.utils.data.DataLoader(train_omni, batch_size=512., pin_memory=True, shuffle=True)

In [11]:
# Find optimal model hyperparameters
n_epochs = 5
if n_epochs > 0:
    deep_kernel.train()
    optimizer = torch.optim.Adam(deep_kernel.gp_layer.parameters(), lr=0.01)
    optimizer.n_iter = 0
    for i in range(n_epochs):
        for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
            train_x_batch = Variable(train_x_batch).cuda()
            train_y_batch = Variable(train_y_batch).cuda()
            optimizer.zero_grad()
            output = deep_kernel(train_x_batch)
            loss = -deep_kernel.gp_layer.marginal_log_likelihood(output, train_y_batch, n_data=len(train_omni))
            kl = deep_kernel.gp_layer.likelihood.kl_div() / len(train_mnist)
            loss = loss + kl
            loss.backward()
            optimizer.n_iter += 1
            print('Iter %d/200 - Loss: %.3f' % (
                i + 1, loss.data[0],
            ))
            optimizer.step()
        """
        deep_kernel.eval()
        test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

        avg = 0.
        i = 0.
        for test_batch_x, test_batch_y in test_data_loader:
            predictions = deep_kernel(Variable(test_batch_x).cuda()).argmax()
            test_batch_y = Variable(test_batch_y).cuda()
            avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
            i += 1.

        print('Score')
        print(avg / i)
        deep_kernel.train()
        """

    torch.save(deep_kernel.state_dict(), '/scratch/bw462/gp_oneshot.dat')
else:
    deep_kernel.load_state_dict(torch.load('/scratch/bw462/gp_oneshot.dat'))

TypeError: softmax() takes exactly 1 argument (2 given)

In [ ]:
train_mnist = torchvision.datasets.MNIST('/scratch/bw462/mnist/', train=True,
                                         download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))                

train_indices = torch.cat([
    train_mnist.train_labels.eq(7).nonzero().squeeze()[:5],
    train_mnist.train_labels.eq(8).nonzero().squeeze()[:5],
    train_mnist.train_labels.eq(9).nonzero().squeeze()[:5],
])
train_mnist.train_data = train_mnist.train_data.index_select(0, train_indices)
train_mnist.train_labels = train_mnist.train_labels.index_select(0, train_indices) - 7

test_mnist = torchvision.datasets.MNIST('/scratch/bw462/mnist/', train=False,
                                        download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

test_indices = test_mnist.test_labels.ge(7).nonzero().squeeze()
test_mnist.test_data = test_mnist.test_data.index_select(0, test_indices)
test_mnist.test_labels = test_mnist.test_labels.index_select(0, test_indices) - 7

len(train_mnist), len(test_mnist)

In [ ]:
oneshot_model = DeepKernel(model).cuda()
oneshot_model.load_state_dict(deep_kernel.state_dict())
oneshot_model.gp_layer.likelihood = gpytorch.likelihoods.SoftmaxGaussianLikelihood(n_features=64, n_classes=10, rank=5).cuda()
oneshot_model.gp_layer.likelihood.feature_mixing_weights.data.copy_(deep_kernel.gp_layer.likelihood.feature_mixing_weights.data)
gp_data_loader = torch.utils.data.DataLoader(train_mnist, batch_size=2048., pin_memory=True, shuffle=True)

In [ ]:
# Find optimal model hyperparameters
oneshot_model.train()
oneshot_model.feature_extractor.eval()
oneshot_model.bottleneck.eval()

optimizer = torch.optim.Adam(oneshot_model.gp_layer.likelihood.parameters(), lr=0.01)
optimizer.n_iter = 0
for i in range(100):
    for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
        train_x_batch = Variable(train_x_batch).cuda()
        train_y_batch = Variable(train_y_batch).cuda()
        optimizer.zero_grad()
        output = oneshot_model(train_x_batch)
        loss = -oneshot_model.gp_layer.marginal_log_likelihood(output, train_y_batch, n_data=len(train_mnist))
        kl = oneshot_model.gp_layer.likelihood.kl_div() / len(train_mnist)
        loss = loss + kl
        loss.backward()
        optimizer.n_iter += 1
        print('Iter %d/200 - Loss: %.3f' % (
            i + 1, loss.data[0],
        ))
        optimizer.step()

oneshot_model.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = oneshot_model(Variable(test_batch_x).cuda()).argmax()
    test_batch_y = Variable(test_batch_y).cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.

print('Score')
print(avg / i)